# Flappy Bird Value Approximator: Linear Regression


## Value Function Approximation
- Function Approximators: Linear Combinations and Neural Network
- Increment Methods: Stochastic Gradient Descent Prediction and Control 
- Batch Methods: Least Squares Prediction and Control, Experience Replay

## Types of Function Approximators
- There are many function Approximators – supervised ML algorithms: Linear combinations of features, Neural Network, Decision Tree, etc.
- RL can get better benefit from differential function Approximators like Linear and Neural Network algorithms
- Incremental methods update the weights on each sample while batch does an updated on each epoch (batch).
- Stochastic Gradient Descent (SCD) is an incremental and iterative optimization algorithm to find values of parameters (weights) of a function that minimizes  cost function. 
- Least Squares method is a form of mathematical regression analysis that finds the line of best fit for a dataset, providing a - visual demonstration of the relationship between the data points.

## Linear Approximating the Q Function
\begin{equation}
\large
Q(s,a) = w_1*f_1(s,a) + w_2*f_2(s,a) + w_n*f_n(s,a)
\end{equation}
![title](images/va_linear.png)


## Q-Learning with Linear Approximation

- Step 1: Start with initial parameter values
- Step 2: Take action a according to an explore/exploit policy, transitioning from s to s’
- Step 3: Perform TD update for each parameter
    \begin{equation}
\large
\theta_i \leftarrow \theta_i + \alpha [R(s) + \beta * max_{a'}\hat{Q_\theta}(s', a') - \hat{Q_\theta}(s, a)]* f_i(s,a)
\end{equation}
- Step 4: Go to Step 2

TD converges close to minimum error solution

Q-Learning can diverge. Converges under some conditions.

## Gradient Descent

Linear function approximation that approximates the underlying value function **V(s)** supposed to be linear; the output would be linear combination of its features. 

Gradient Descent is one of the simplest form of linear function approximation. In gradient descent methods, the feature vector is a column vector which is expected to be differentiable function. Gradient Descent methods minimize error on the observed samples by adjusting the parameters after each sample by a small amount in the direction that would reduce the error between the actual and expected. However in Reinforcement Learning with continuous space like flappy bird, gradient descent, does not guarantee the convergence. 


## Radial Basis Function (RBF)

Radial Basis Function is useful for Reinforcement Environments with continuous state space and ideal for flappy bird like environments where the number of features are small. Radial Basis an extension to tile encoding where the the state space (features) is grouped into tiles where each tile represents a binary feature (0 or 1). In RBF, each feature can have have any value between 0 and 1, instead of 0 or 1. Due to this RBF can generate better approximate functions that are smoothly differentiable and also are easy to calculate. Hence it can converge better compared to Gradient Descent or Tile based value approximators. RBF can get complex for non-linear features. RBF can hold the key for reinforcement learning given that it does not discard the previously learned information while gathering the new information.



# Linear Regression Implementation

Jupyter notebook function to disable cell-level scrolling

In [1]:
%%javascript
/*jupyter functionality - remove the cell-level scrollbar*/

IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
#%run plotting.py
%run flappy_bird_env_open_ai_gym.py #open AI gym clone


In [3]:
%matplotlib inline
import os
import datetime
import gym
import itertools
import matplotlib
import numpy as np
import sys
import sklearn.pipeline
import sklearn.preprocessing
import json
import random
import collections

if "../" not in sys.path:
    sys.path.append("../") 

#from lib import plotting
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_approximation import RBFSampler

matplotlib.style.use('ggplot')



In [4]:
env = FlappyBirdEnv()

In [5]:
# Feature Preprocessing: Normalize to zero mean and unit variance
# We use a few samples from the observation space to do this
sample_size = 100000 #10000
observation_examples = np.array([env.observation_space.sample() for x in range(sample_size)])
#observation_examples = states
print(observation_examples.shape)
#print(observation_examples)

scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(observation_examples)

(100000, 3)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [6]:
#Used to convert a state to a featurized representation.
# We use RBF kernels with different variances to cover different parts of the space
featurizer = sklearn.pipeline.FeatureUnion([
        ("rbf1", RBFSampler(gamma=5.0, n_components=100)),
        ("rbf2", RBFSampler(gamma=2.0, n_components=100)),
        ("rbf3", RBFSampler(gamma=1.0, n_components=100)),
        ("rbf4", RBFSampler(gamma=0.5, n_components=100))
        ])
featurizer.fit(scaler.transform(observation_examples))

FeatureUnion(n_jobs=1,
       transformer_list=[('rbf1', RBFSampler(gamma=5.0, n_components=100, random_state=None)), ('rbf2', RBFSampler(gamma=2.0, n_components=100, random_state=None)), ('rbf3', RBFSampler(gamma=1.0, n_components=100, random_state=None)), ('rbf4', RBFSampler(gamma=0.5, n_components=100, random_state=None))],
       transformer_weights=None)

In [7]:
class Estimator():
    """
    Value Function approximator. 
    """
    
    def __init__(self):
        # We create a separate model for each action in the environment's
        # action space. Alternatively we could somehow encode the action
        # into the features, but this way it's easier to code up.
        self.models = []
        for _ in range(env.action_space.n):
            model = SGDRegressor(learning_rate="constant")
            # We need to call partial_fit once to initialize the model
            # or we get a NotFittedError when trying to make a prediction
            # This is quite hacky.
            model.partial_fit([self.featurize_state(env.reset(2))], [0])
            self.models.append(model)
    
    def featurize_state(self, state):
        """
        Returns the featurized representation for a state.
        """               
        scaled = scaler.transform([state])       
        featurized = featurizer.transform(scaled)        
        return featurized[0]       
    
    def predict(self, s, a=None):
        """
        Makes value function predictions.
        
        Args:
            s: state to make a prediction for
            a: (Optional) action to make a prediction for
            
        Returns
            If an action a is given this returns a single number as the prediction.
            If no action is given this returns a vector or predictions for all actions
            in the environment where pred[i] is the prediction for action i.
            
        """
                
        # TODO: Implement this!
        if a is not None:
            prediction = self.models[a].predict([self.featurize_state(s)])
            return prediction[0]
        
        else:
            predictions = np.array([self.models[i].predict([self.featurize_state(s)]) for i in range(env.action_space.n)])
            return predictions.reshape(-1)
            
    def update(self, s, a, y):
        """
        Updates the estimator parameters for a given state and action towards
        the target y.
        """
        # TODO: Implement this!
        self.models[a].partial_fit([self.featurize_state(s)], [y])

In [8]:
def make_epsilon_greedy_policy(estimator, epsilon, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function approximator and epsilon.
    
    Args:
        estimator: An estimator that returns q values for a given state
        epsilon: The probability to select a random action . float between 0 and 1.
        nA: Number of actions in the environment.
    
    Returns:
        A function that takes the observation as an argument and returns
        the probabilities for each action in the form of a numpy array of length nA.
    
    """
    def policy_fn(observation):
        A = np.ones(nA, dtype=float) * epsilon / nA
        q_values = estimator.predict(observation)
        best_action = np.argmax(q_values)
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn

**save_stats method is used to capture the output of all the episodes with metrics: algorithm, duration, episode, reward and score.**


In [9]:
#start  = datetime.datetime.now()
#data = []
def save_stats(episode,data,score):

    algorithm = "linear"
    
#    data.append(json.dumps({ "algorithm": algorithm, 
#                "duration":  "{}".format(duration), 
#                "episode":   episode, 
#                "reward":    reward, 
#                "score":     score}))
        
    if (score >= 50):
        print(data)

    

    file_name = 'data/stats_flappy_bird_{}.json'.format(algorithm)
            
            # delete the old file before saving data for this session
    if episode == 0 and os.path.exists(file_name): os.remove(file_name)
                
            # open the file in append mode to add more json data
    file = open(file_name, 'a+')  
    for item in data:
        file.write(item)  
        file.write(",")
            #end for
    file.close()
            
        #    data = []
        #end if   

In [10]:
def q_learning(env, estimator, num_episodes, discount_factor=0.99, epsilon=0.3, epsilon_decay=0.0001):
    """
    Q-Learning algorithm for off-policy TD control using Function Approximation.
    Finds the optimal greedy policy while following an epsilon-greedy policy.
    
    Args:
        env: OpenAI environment.
        estimator: Action-Value function estimator
        num_episodes: Number of episodes to run for.
        discount_factor: Gamma discount factor.
        epsilon: Chance the sample a random action. Float betwen 0 and 1.
        epsilon_decay: Each episode, epsilon is decayed by this factor
    
    Returns:
        An EpisodeStats object with two numpy arrays for episode_lengths and episode_rewards.
    """
    start = datetime.datetime.now()
    data = []
    nA = env.action_space.n
    algorithm = "linear"
    EpisodeStats = collections.namedtuple("Stats",["episode_lengths", "episode_rewards"])
    
    # Keeps track of useful statistics
    stats = EpisodeStats(
        episode_lengths=np.zeros(num_episodes),
        episode_rewards=np.zeros(num_episodes))    
    
    episode_reward = 0
    for i_episode in range(num_episodes):
        
        # The policy we're following
        policy = make_epsilon_greedy_policy(
            estimator, epsilon * epsilon_decay**i_episode, nA)
        
        # Print out which episode we're on, useful for debugging.
        # Also print reward for last episode
        last_reward = stats.episode_rewards[i_episode - 1]
        #print("\rEpisode {}/{} ({})".format(i_episode + 1, num_episodes, last_reward))
        sys.stdout.flush()
        
        episode_reward += last_reward
        
        # TODO: Implement this!
        state = env.reset(2)
        
        for t in itertools.count():
            
            # sample the action from the epsilon greedy policy
            action = np.random.choice(nA, p=policy(state))
            # Perform the action -> Get the reward and observe the next state
            new_state, reward, terminated, _ = env.step(action,2)
            env.render()
            new_action = np.random.choice(nA, p=policy(new_state))
                        
            #stats.episode_rewards[i_episode] += reward
            #stats.episode_lengths[i_episode] = t
            
            q_values_new_state = estimator.predict(new_state)

            # value that we should have got
            # The Q-learning target policy is a greedy one, hence the `max`
            td_target = reward + discount_factor * np.max(q_values_new_state)
            estimator.update(state, action, td_target)            
            
            # update current state
            state = new_state
        
            if terminated:
                break
        
        duration = datetime.datetime.now() - start 
        data.append(json.dumps({ "algorithm": algorithm, 
                "duration":  "{}".format(duration), 
                "episode":   i_episode, 
                "reward":    episode_reward, 
                "score":     env.score}))
        
        if (i_episode% 500 == 0):
            save_stats(i_episode,data,env.score)
    
    return stats

In [11]:
estimator = Estimator()

C:\Users\cheru\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [ ]:
max_episodes = 25000
if __name__ =="__main__":
    stats = q_learning(env, estimator, max_episodes, epsilon=0.1)

        

C:\Users\cheru\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
